# Problem Description
Variance swap on SPX with:
- vega notional = 1
- vol strike = 20%
- period from 1-Nov-2022 to 30-Nov-2022
- rf rate = 5%
- vol env = 15%

Task:
- Swap price at 9-Nov 2022?
- Calculate the greeks: Delta, Gamma, Vega, Rho, Theta

Approaches:
- Monte Carlo under:
    - Geometric Brownian Motion
    - Heston Stochastic Volatility Model
- Analytical approach

Task 2:
- relation betweeen the errors (from Monte Carlo Approaches) and number of simulations

Additional Task: 
- extension to the model accounting for realistic market dynamics

# Relevant information for the concepts:

## Concepts
- Variance swap - betting on how much the market will move no matter the direction
    - vega notional - scaling factor for variance - need to convert it to variance notional 
        - variance notional = vega notional / (2 * sigma strike)
            - means the dollar size of the payoff
    - volatility strike (sigma strike) - annualized the strike of the swap
        - if realized vol > 20 -> buyer wins 
        - else if realized vol < 20 -> seller wins 
        - realized vol = 20 -> expires worthless 
    - period - the period used to calcualte the payoff
        - European (modeled in this task) - caluclate the payoff at the end of the period 
        - American (harder to model) - option can be settled at any time in the period 
    - rf rate 
        - used to discount the future payout from 30-Nov-2022 back to 9-Nov-2022
        - in GBM it is used for the drift
            - in this case when we simulate changes in the log returns (log because we want to sum them easily instead of multiplying sequentually normal returns)
    - volatility environment (sigma env) - annualized volatility of spx 

- The greeks: - they are all sensitivites of the price of the swap 
    - delta - sensitivity to the underlying asset price - first derivative of the price of the swap with respect to price of the asset 
        - analytical approach - 
    - gamma - calcualtes how fast the delta is changing - first derivative of delta with respect to the price of the asset or second derivative of the price of the swap with respect to the price of the asset
        - for calucaltion is used Taylor Series Expansion
    - vega - sensitivity to volatility - first derivative of the price of the swap with respect to volatility 
    - rho - sensitivity to the risk free rate - first derivative of the price with respect to the risk free rate
    - theta - sensitivity to the passage of time - first derivative of the price with respest to the time  


## Soluion:
The value of the swap is calculated with 3 times. All three fallow the same logic but they use different methods to calculate the future variance.
There are small differences in the caluclation of the greeks therefore their calcualtion will be described for each approach separately. By definition the variance swa is not sensitive to the price of the asset there fore the delta and gamma are 0. 
- Delta - shows sensititvy of the price of the swap to the price of the asset - first derivative of the price of the swap with respect to price of the asset
- Gamma - shows how fast the delta is changing - first derivative of delta with respect to the price of the asset or second derivative of the price of the swap with respect to the price of the asset
 
### Base logic for the value calculation: 
- First calculate variance before and after valuation date (9-Nov-2022)
    - calculate daily expected variance as annual variance divided by the number of days in the business year (because it is valience is changing only when the market is open)
    - calculate the expected variance for the period by multiplying the expected variance by the number of days in the period 
- calculate total realized variance as the weighted sum of the past and future variance (weights are the number of days from each period)
- calculate the payoff = variance notional * (total realized variance - sigma strike ^ 2)
    - variance notional - derived from vega notional
    - total realized variance - sigma strike ^ 2 - this is the difference between the expected and the realized variance 
- calucalte the discount factor -> exp ^ ((-1) * risk free rate * time to maturity )
    - time to maturity -> diff from valuation date to maturity date / number of days in year (365)
        - 365 and not 252 because money accumulates interest every day 
            - 252 is for the volatility because it moves only on market days while the interest is accumulated every day no metter th market 
        - exponent represent the continuous compounding -> it is not the same as compounding every day and is used because it gives smoother function for the price
- calcualte price of the swap for the evaluation date (9-Nov-2022)
    - price = discount factor * payoff 

Note: if enough time make the calucaltion of the past variance with data from yahoo finance

### Approaches:
- Analytical Base approach: 
    - Future variance - just calculate it as it is calculated for the past 
    - Greeks calculation with analytical approach:
        - delta and gamma - they depend on the price of spx therefore are 0 because we are not using the price of the asset 
        - rho - sensitivity to the risk free rate - first derivative of the price with respect to the risk free rate
            - calucalted as rhp = days to mat / 365 * price 
        - vega - sensitivity to volatility - first derivative of the price of the swap with respect to volatility 
            - calculate discount factor -> exp ^ ((-1) * risk free rate * time to maturity )
            - calculate time weight - left days as percentace of the whole period 
            - calculate the derivative of the variance with respect to volatility - first derivative of sigma strike ^ 2 -> 2 * sigma strike 
            - calculate vega -> discount factor * time weight * variance notional * derivative of the variance with respect to volatility
        - theta - sensitivity to the passage of time - first derivative of the price with respest to the time
            - calucalte value for today and for tomorrow 
            - calculate the difference and divide by the number of days between today and tomorrow (1 so dont divide at all)


### Monte Carlo under GBM
- Future variance is calculated with Monte Carlo under GBM - assumes movement based on drift and randomness sclaed by time step -> d(ln(St)) = drift  + diffusion * randomness (Z)
    - calc drift and diffusion
        - drift - this is the trend of the returns 
            - calculated as -> rf rate - Itos correction * dt
                - Itos correction = (1/2*(vol env)^2) 
                    - this accounts for the jagged path of the returns 
                - dt to scale down the annual rates 
        - diffusion - magnitute of the randomness 
            - diffusion = vol env * sqrt(dt) 
                - vol env - determines how big the swing will be 
                - sqrt(dt) - standard rule for scaling uncertainty: 
                    - trends - scales with time -> dt - trends is linear 
                    - risk - scales with sqrt(dt) - uncertainty is parabolic  
    - generate randomness (Z) -  np.random.normal(0, 1, (simulations, days left to maturity))
    - calc future returns = drift + diffusion * Z
    - future variance = sum the returns ^ 2 -> because we dont care what is the direction only how much is the movement 

- Greeks calculation with Monte Carlo under GBM. Very important to note that the generated random noise is the same for all the 
    - delta and gamma - they depend on the price of spx therefore are 0 because we not using the price of the asset 
    - For rho, vega calculate the values of the swap with small changes in the parametes that they are sensitive to. The rho and vega values are the differences between the  the original swap value and the new values calculated after the change of the specific parameter. The parameters are: 
        - rho - risk free rate 
        - vega - volatility 
    - For theta make one new swap with changed maturity (10-Nov-2022) and calculates its price. The theta is the difference between the prices of the original swap and the new swap.

### Heston Stochastic Volatility Model:
This model adds randomess to the volatility environment (sigma env). There are two approaches to calculate the future variance - analytical and monte carlo. We are calcualting the analitical to be able to compare the results with the monte carlo approach. This approach need three new parameters. The values given in the example are popular:
    - kappa = 2 -> how fast the variance reverts to the long-run level 1 means too slow 20 means too fast.
    - vol of var = 0.3 -> volatility of variance 
    - rho = -0.7 -> correlation between the returns and the variance 
    - theta - (this is new but not used in this example) long-run level of the variance in this exmaple it is set to the current enviroment variance 

#### Heston Analytical approach:
- Future variance is calculated with the analytical approach we use the closed form solution for the future variance. It is calculated as the sum of long term anchor and shock:
    - long term anchor - time to matiruty in years * theta -> if we wait a one year, we expect the variance to be the long term variance 
    - shock - the random component of the future variance 
        - shock = (diff in variance * decay facotr) / kappa
            - diff in variance = init variance - long term variance (theta)
            - decay factor = exp ^ (-kappa * time to matiruty) -> how fast the variance goes to the mean 

- Greeks calculation with Heston Analytical approach are all calucated as the greeks in the monte carlo approach but using the analytical Heston model to calculate the future variance for the values with changed parameters. 

#### Heston Monte Carlo approach:
- Future variance is calculated under Monte Carlo approach 
    - First is calucated a correlation complement = sqrt(1 - rho ^ 2)
        - Additional information -> This correlation complement comes from Cholesky Decomposition, it shows how much the Stock depends on itself while rho shows how much it depends on the variance.
            - Cholesky Decomposition - with lieanr algebra look for matrix that when multiplied by transpose of itself gives the correlation matrix that is [[1, rho], [rho, 1]] -> the matrix is [[1, rho], [0, corr_compl]]
    - For each of the days left to maturity we need to calucalte the variance and update the variance value. The calucaltion diffusion part of the calculation of the future variance is more complex as it includes the correlation complement. Full description of the calculation of both the returns and the variance is in the code.
            
- Greeks calculation with Heston Monte Carlo approach are all calucated as the greeks in the monte carlo approach but using the Heston Monte Carlo model to calculate the future variance for the values with changed parameters. Very important to note that the generated random noises for the returns and the variance are the same for the calcutaion of new swap values. 
    

# Solution


To install dependencies, run the following command:
> `pip install -r requirements.txt`

In [1]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"

In [2]:
from utils import run_analytical_results, run_gbm_mc_greeks, create_variance_swap, run_heston_analytical_results, run_heston_mc_greeks
data = create_variance_swap()

### Constant volatility solution

Analytical calucaltion of the price of the variance swap and the greeks for 9-Nov-2022

In [3]:
analytical_res = run_analytical_results(data)
analytical_res

{'price': np.float64(-0.04362432468814453),
 'delta': 0,
 'gamma': 0,
 'rho': np.float64(0.0025098926532905073),
 'vega': np.float64(0.4985637107216515),
 'theta': np.float64(-5.976344218082408e-06)}

Monte Carlo calucaltion under Geometric Brownian Motion calcualtion the price of the variance swap and the greeks for 9-Nov-2022

In [4]:
gbm_mc = run_gbm_mc_greeks(data, n_simulations=50_000)
gbm_mc 

{'price': np.float64(-0.043659679490256724),
 'delta': 0,
 'gamma': 0,
 'rho': np.float64(0.003005524677884841),
 'vega': np.float64(0.4978907619251141),
 'theta': np.float64(-3.115904879301412e-06)}

Calculation of the error of the price and greeks with different numbers of simulations

In [5]:
sim_steps, results =np.linspace(1_000, 500_000, 25, dtype=int), []

for n in sim_steps:
    mc_res = run_gbm_mc_greeks(data, n)
    row_res = {"n_simulations": n}
    for metric in ['price', 'rho', 'vega', 'theta']:
        true_val = analytical_res[metric]
        mc_val = mc_res[metric]
        abs_err = abs(mc_val - true_val)
    
        pct_err = abs_err / abs(true_val) if abs(true_val) > 1e-12 else 0.0
        row_res[metric + "_abs_err"] = abs_err
        row_res[metric + "_pct_err"] = pct_err
    results.append(row_res)

errors_df = pd.DataFrame(results)

In [6]:
errors_df

,n_simulations,price_abs_err,price_pct_err,rho_abs_err,rho_pct_err,vega_abs_err,vega_pct_err,theta_abs_err,theta_pct_err
0,1000,1.702938e-04,0.003904,0.000480,0.191271,0.002071,0.004153,8.741250e-05,14.626417
1,21791,1.491218e-04,0.003418,0.000478,0.190491,0.001790,0.003589,5.541547e-05,9.272470
2,42583,3.431322e-05,0.000787,0.000535,0.212995,0.000238,0.000478,1.228389e-05,2.055419
3,63375,2.929403e-05,0.000672,0.000533,0.212406,0.000172,0.000345,1.820259e-05,3.045773
4,84166,5.566029e-05,0.001276,0.000472,0.187897,0.000548,0.001100,9.959868e-06,1.666549
5,104958,3.015812e-06,0.000069,0.000544,0.216788,0.000182,0.000365,1.988754e-06,0.332771
6,125750,6.140569e-06,0.000141,0.000538,0.214492,0.000138,0.000277,2.854752e-07,0.047768
7,146541,5.848456e-05,0.001341,0.000548,0.218510,0.001002,0.002011,9.816499e-06,1.642559
8,167333,4.410145e-05,0.001011,0.000515,0.205303,0.000797,0.001599,7.208332e-07,0.120614
9,188125,3.549953e-05,0.000814,0.000549,0.218810,0.000697,0.001398,7.099048e-06,1.187858


**Plot of the absolute error of the price for each number of simulations**
As expected the error decreases as the number of simulations increases because the mean becomes more stable. 

In [7]:
errors_df.set_index("n_simulations")['price_abs_err'].plot()

In [8]:
# interactive plotly plot of all errors - you can click on the legend to hide/show the ploted lines
errors_df.set_index("n_simulations").plot()

### Non-constant volatility solution with Heston Stochastic Volatility Model

Analytical Heston Model calculation of the price of the variance swap and the greeks for 9-Nov-2022

In [9]:
heston_analytical_res = run_heston_analytical_results(data)
heston_analytical_res

{'price': np.float64(-0.04362432468814453),
 'delta': 0,
 'gamma': 0,
 'rho': np.float64(0.0025098926532615695),
 'vega': np.float64(0.471863703879468),
 'theta': np.float64(-5.976344218082408e-06)}

Monte Carlo under Heston Model calculation of the price of the variance swap and the greeks for 9-Nov-2022

In [10]:
heston_mc_res = run_heston_mc_greeks(data,n_simulations=50_000)
heston_mc_res

{'price': np.float64(-0.04362487438227468),
 'delta': 0,
 'gamma': 0,
 'rho': np.float64(0.0030752532736988414),
 'vega': np.float64(0.4733801081830899),
 'theta': np.float64(-8.425109465005909e-06)}

Calculation of the error of the price and greeks with different numbers of simulations

In [11]:
heston_res = []
for n in sim_steps:
    mc_res = run_heston_mc_greeks(data, n)
    row_res = {"n_simulations": n}
    
    for metric in ['price', 'rho', 'vega', 'theta']:
        true_val = heston_analytical_res[metric]
        mc_val = mc_res[metric]
        abs_err = abs(mc_val - true_val)
        
        pct_err = abs_err / abs(true_val) if abs(true_val) > 1e-9 else 0.0
        
        row_res[metric + "_abs_err"] = abs_err
        row_res[metric + "_pct_err"] = pct_err
        
    heston_res.append(row_res)

heston_errors_df = pd.DataFrame(heston_res)

In [12]:
heston_errors_df

,n_simulations,price_abs_err,price_pct_err,rho_abs_err,rho_pct_err,vega_abs_err,vega_pct_err,theta_abs_err,theta_pct_err
0,1000,0.000333,0.007634,0.000625,0.248910,0.006223,0.013188,1.251105e-04,20.934282
1,21791,0.000158,0.003627,0.000563,0.224123,0.000157,0.000333,5.270813e-05,8.819460
2,42583,0.000053,0.001206,0.000506,0.201640,0.000769,0.001630,5.763047e-05,9.643098
3,63375,0.000128,0.002932,0.000477,0.189983,0.002981,0.006317,6.342854e-07,0.106133
4,84166,0.000022,0.000515,0.000495,0.197105,0.001857,0.003935,6.309087e-07,0.105568
5,104958,0.000047,0.001069,0.000557,0.221757,0.001099,0.002329,9.720164e-06,1.626440
6,125750,0.000039,0.000883,0.000499,0.198634,0.001177,0.002494,8.727628e-06,1.460362
7,146541,0.000040,0.000917,0.000518,0.206438,0.001957,0.004147,1.196095e-05,2.001383
8,167333,0.000039,0.000886,0.000471,0.187656,0.002049,0.004343,3.204344e-06,0.536171
9,188125,0.000045,0.001036,0.000509,0.202813,0.001101,0.002334,1.395328e-05,2.334751


In [13]:
heston_errors_df.set_index("n_simulations")["price_abs_err"].plot()

In [14]:
heston_errors_df.set_index("n_simulations").plot()